In [1]:
import regex as re
import pandas as pd

In [2]:
with open("uu8-1999.txt") as file:
    file = file.read()

## Remove Header

In [3]:
file_no_header = re.sub("\\nPRESIDEN\\nREPUBLIK INDONESIA\\n- \p{N}+ -", " ", file, flags=re.DOTALL)
file_no_header = re.sub("(\\n[^\\n]*? \…[^\\n]*)(?=\\n)", " ", file_no_header, flags=re.DOTALL)

## Pisah Penjelasan

In [4]:
penjelasan = re.search("PRESIDEN\\nREPUBLIK INDONESIA\\nPENJELASAN.*", file_no_header, re.DOTALL).group()
badan = file_no_header.replace(penjelasan, "")

# Badan

## Ambil BAB

In [5]:
def ambil_bab(text):
    pattern = "(?<=\\n)(BAB [IVXLCDM]+.*?)(?=\\nBAB|$)"
    bab = re.findall(pattern, text, re.DOTALL)
    if bab:
        ## ambil nomor dan nama bab
        pattern_no = "(?<=BAB)(?:\s*)([IVXLCDM]+)"
        pattern_nama = "(?<=BAB)(?:\s*[IVXLCDM]+\s*\\n)([\p{Lu}\s\\n]*)(?=\\n)"
        pattern_isi = "(?<=BAB)(?:\s*[IVXLCDM]+\s*\\n[\p{Lu}\s\\n]*)(?:\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) for i in bab]
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) for i in bab]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) for i in bab]
        return no, nama, isi
    else:
        no = [""]
        nama = [""]
        isi = [text]
        return no, nama, isi

Penjelasan
1. (?<=\\n)(BAB [IVXLCDM]+.*?)(?=\\nBAB|$)<br>
Digunakan untuk mengambil semua bab<br>
Kondisi:<br>
    a. Text dimulai dengan \n tanpa mengambil \n<br>
    b. Setelah \n dimulai dengan BAB(spasi)[Bilangan Romawi 1 digit dapat diulang beberapa kali] contoh (BAB IIV, BAB VVIX) akan diambil<br>
       BILANGAN ROMAWI YANG TIDAK VALID DAPAT DIAMBIL<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. pengambilan karakter akan berhenti ketika bertemu \nBAB<br>

2. (?<=BAB)(?:\s*)([IVXLCDM]+)<br>
Digunakan untuk mengambil no bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas [Bilangan romawi 1 digit dapat diulang beberapa kali] akan diambil<br>
       BILANGAN ROMAWI YANG TIDAK VALID DAPAT DIAMBIL<br>

3. (?<=BAB)(?:\s*[IVXLCDM]+\s*\\n)([\p{Lu}\s\\n]*)(?=\\n)<br>
Digunakan untuk mengambil nama bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali)[Bilangan Romawi 1 digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n<br>
    b. Mengambil semua karakter berupa huruf kapital atau spasi atau \n<br>
    c. Pengambilan karakter akan berhenti ketika bertemu \n<br>

4. (?<=BAB)(?:\s*[IVXLCDM]+\s*\\n[\p{Lu}\s\\n]*)(?:\\n)(.*)<br>
Digunakan untuk mengambil isi bab<br>
Kondisi:<br>
    a. Text dimulai dengan BAB(spasi dapat diulang beberapa kali)[Bilangan Romawi 1 digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n<br>
    b. Setelah kondisi di atas teks dimulai dengan huruf kapital atau spasi atau \n hingga berhenti pada \n tanpa diambil (nama bab)<br>
    c. Semua karakter sesudahnya diambil<br>

In [6]:
no_bab, nama_bab, isi_bab = ambil_bab(badan)
df_bab = {"no_bab": no_bab, "nama_bab": nama_bab, "isi_bab": isi_bab}

In [7]:
df_bab = pd.DataFrame(df_bab)
df_bab

,no_bab,nama_bab,isi_bab
0,I,KETENTUAN UMUM,Pasal 1\nDalam Undang-undang ini yang dimaksud...
1,II,ASAS DAN TUJUAN,Pasal 2\nPerlindungan konsumen berasaskan manf...
2,III,HAK DAN KEWAJIBAN,Bagian Pertama\nHak dan Kewajiban Konsumen\nPa...
3,IV,PERBUATAN YANG DILARANG\nBAGI PELAKU USAHA,Pasal 8\n(1) Pelaku usaha dilarang memproduksi...
4,V,KETENTUAN PENCANTUMAN KLAUSULA BAKU,Pasal 18\n(1) Pelaku usaha dalam menawarkan ba...
5,VI,TANGGUNG JAWAB PELAKU USAHA,Pasal 19\n(1) Pelaku usaha bertanggung jawab m...
6,VII,PEMBINAAN DAN PENGAWASAN,Bagian Pertama\nPembinaan\nPasal 29\n(1) Pemer...
7,VIII,BADAN PERLINDUNGAN KONSUMEN NASIONAL,"Bagian Pertama \nNama, Kedudukan, Fungsi, dan ..."
8,IX,LEMBAGA PERLINDUNGAN KONSUMEN\nSWADAYA MASYARAKAT,Pasal 44\n(1) Pemerintah mengakui lembaga perl...
9,X,PENYELESAIAN SENGKETA,Bagian Pertama\nUmum\nPasal 45\n(1) Setiap kon...


## Ambil Bagian

In [8]:
def ambil_bagian(text):
    pattern = "(?<=\\n|^)(Bagian \p{Lu}.*?)(?=\\nBagian \p{Lu}|$)"
    bagian = re.findall(pattern, text, re.DOTALL)
    if bagian:
        pattern_nama = "(?<=Bagian \p{Lu})(?:\p{L}*\s*\\n)(.*?)(?=\\nPasal)"
        pattern_isi = "Pasal.*"
        nama = [re.search(pattern_nama, i, re.DOTALL).group(1) for i in bagian]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) for i in bagian]
        return nama, isi
    else:
        nama = [""]
        isi = [text]
        return nama, isi

Penjelasan
1. (?<=\\n|^)(Bagian \p{Lu}.*?)(?=\\nBagian \p{Lu}|$)<br>
Digunakan untuk mengambil semua bagian<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan Bagian(spasi)[Huruf kapital] contoh (Bagian H, Bagian C, Bagian D) akan diambil<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. pengambilan karakter akan berhenti ketika bertemu \nBagian(spasi)[Huruf kapital]<br>

2. (?<=Bagian \p{Lu})(?:\p{L}*\s*\\n)(.*?)(?=\\nPasal)<br>
Digunakan untuk mengambil nama bagian<br>
Kondisi:<br>
    a. Text dimulai dengan Bagian(spasi)[Huruf kapital][Huruf atau spasi dapat diulang beberapa kali]\n tanpa diambil<br>
    b. Semua karakter selanjutnya akan diambil<br>
    c. Pengambilan karakter akan berhenti ketika bertemu \nPasal<br>

3. Pasal.*<br>
Digunakan untuk mengambil isi bagian<br>
Kondisi:<br>
    a. Text dimulai dengan Pasal dengan diambil<br>
    b. Semua karakter selanjutnya akan diambil<br>

In [9]:
for i in range(len(df_bab)):
    nama_bagian, isi_bagian = ambil_bagian(df_bab["isi_bab"][i])
    df_bagian = {"nama_bagian": nama_bagian, "isi_bagian": isi_bagian}
    display(pd.DataFrame(df_bagian))

,nama_bagian,isi_bagian
0,,Pasal 1\nDalam Undang-undang ini yang dimaksud...


,nama_bagian,isi_bagian
0,,Pasal 2\nPerlindungan konsumen berasaskan manf...


,nama_bagian,isi_bagian
0,Hak dan Kewajiban Konsumen,Pasal 4\nHak konsumen adalah:\na. hak atas ken...
1,Hak dan Kewajiban Pelaku Usaha,Pasal 6\nHak pelaku usaha adalah:\na. hak untu...


,nama_bagian,isi_bagian
0,,Pasal 8\n(1) Pelaku usaha dilarang memproduksi...


,nama_bagian,isi_bagian
0,,Pasal 18\n(1) Pelaku usaha dalam menawarkan ba...


...

,nama_bagian,isi_bagian
0,,Pasal 49\n(1) Pemerintah membentuk badan penye...


,nama_bagian,isi_bagian
0,,Pasal 59\n(1) Selain Pejabat Polisi Negara Rep...


,nama_bagian,isi_bagian
0,Sanksi Administratif,Pasal 60\n(1) Badan penyelesaian sengketa kons...
1,Sanksi Pidana,Pasal 61\nPenuntutan pidana dapat dilakukan te...


,nama_bagian,isi_bagian
0,,Pasal 64\nSegala ketentuan peraturan perundang...


,nama_bagian,isi_bagian
0,,Pasal 65\nUndang-undang ini mulai berlaku sete...


In [10]:
nama_bagian, isi_bagian = ambil_bagian(df_bab["isi_bab"][7])
df_bagian = {"nama_bagian": nama_bagian, "isi_bagian": isi_bagian}
pd.DataFrame(df_bagian)

,nama_bagian,isi_bagian
0,"Nama, Kedudukan, Fungsi, dan Tugas",Pasal 31\nDalam rangka mengembangkan upaya per...
1,Susunan Organisasi dan Keanggotaan,Pasal 35\n(1) Badan Perlindungan Konsumen Nasi...


## Ambil Pasal

In [11]:
def ambil_pasal(text):
    pattern = "(?<=\\n|^)(Pasal \p{N}+\s*\\n.*?)(?=\\nPasal|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)(\p{N}+)"
        pattern_isi = "(?<=Pasal)(?:\s*\p{N}+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) for i in pasal]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) for i in pasal]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

Penjelasan
1. (?<=\\n|^)(Pasal \p{N}+\s*\\n.*?)(?=\\nPasal|$)<br>
Digunakan untuk mengambil semua pasal<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan Pasal(spasi)[Nomor satu digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n contoh (Pasal 1\n, Pasal 23     \n) akan diambil<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>
    d. Pengambilan karakter akan berhenti ketika bertemu \nPasal atau (end of string)<br>

2. (?<=Pasal)(?:\s*)(\p{N}+)<br>
Digunakan untuk mengambil no pasal<br>
Kondisi:<br>
    a. Text dimulai dengan Pasal(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan [Nomor satu digit dapat diulang beberapa kali] akan diambil<br>

3. (?<=Pasal)(?:\s*\p{N}+\s*\\n)(.*)<br>
Digunakan untuk mengambil isi pasal<br>
Kondisi:<br>
    a. Teks dimulai dengan Pasal(spasi dapat diulang beberapa kali) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai dengan [Nomor satu digit dapat diulang beberapa kali](spasi dapat diulang beberapa kali)\n akan diambil (nomor pasal)<br>
    c. Setelah kondisi diatas akan mengambil semua karakter<br>

In [12]:
for i in range(len(df_bagian)):
    no_pasal, isi_pasal = ambil_pasal(df_bagian["isi_bagian"][i])
    df_pasal = {"no_pasal": no_pasal, "isi_pasal": isi_pasal}
    display(pd.DataFrame(df_pasal))

,no_pasal,isi_pasal
0,31,Dalam rangka mengembangkan upaya perlindungan ...
1,32,Badan Perlindungan Konsumen Nasional berkedudu...
2,33,Badan Perlindungan Konsumen Nasional mempunyai...
3,34,(1) Untuk menjalankan fungsi sebgaimana dimaks...


,no_pasal,isi_pasal
0,35,(1) Badan Perlindungan Konsumen Nasional terdi...
1,36,Anggota Badan Perlindungan Konsumen Nasional t...
2,37,Persyaratan keanggotaan Badan Perlindungan Kon...
3,38,Keanggotaan Badan Perlindungan Konsumen Nasion...
4,39,"(1) Untuk kelancaran pelaksanaan tugas, Badan ..."
5,40,(1) Apabila diperlukan Badan Perlindungan Kons...
6,41,"Dalam pelaksanaan tugas, Badan Perlindungan Ko..."
7,42,Biaya untuk pelaksanaan tugas Badan Perlindung...
8,43,Ketentuan lebih lanjut mengenai pembentukan Ba...


In [13]:
no_pasal, isi_pasal = ambil_pasal(df_bagian["isi_bagian"][0])
df_pasal = {"no_pasal": no_pasal, "isi_pasal": isi_pasal}

In [14]:
df_pasal = pd.DataFrame(df_pasal)
df_pasal

,no_pasal,isi_pasal
0,31,Dalam rangka mengembangkan upaya perlindungan ...
1,32,Badan Perlindungan Konsumen Nasional berkedudu...
2,33,Badan Perlindungan Konsumen Nasional mempunyai...
3,34,(1) Untuk menjalankan fungsi sebgaimana dimaks...


## Ambil Ayat

In [15]:
def ambil_ayat(text):
    clean_text = re.sub('(?<=ayat )(\\n)(?=\(\p{N}\))', "", text, flags=re.DOTALL)
    ayat = re.findall('(?<=\\n|^)(\(\p{N}+\).*?)(?=\\n\(\p{N}\)|$)', clean_text, flags=re.DOTALL)
    
    if ayat:
        pattern_no = "(?<=\\n|^)(?:\()(\p{N}+)(?=\))"
        pattern_isi = "(?<=\(\p{N}+\)\s*)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) for i in ayat]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(0) for i in ayat]
        
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

Penjelasan
1. (?<=\\n|^)(\(\p{N}+\).*?)(?=\\n\(\p{N}\)|$)<br>
Digunakan untuk mengambil semua ayat<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai oleh (buka kurung)[nomor dapat diulang beberapa kali](tutup kurung)<br>
    c. Setelah kondisi diatas semua karakter diambil<br>
    d. Pengambilan karakter diambil hingga bertemu \n(buka kurung)[nomor dapat diulang beberapa kali](tutup kurung) atau (end of string)<br>

2. (?<=\\n|^)(?:\()(\p{N}+)(?=\))<br>
Digunakan untuk mengambil no ayat<br>
Kondisi:<br>
    a. Text dimulai dengan \n atau (start of string) tanpa diambil<br>
    b. Setelah kondisi diatas dimulai oleh (buka kurung) tanpa diambil<br>
    c. Setelah kondisi diatas dimulai oleh [nomor dapat diulang beberapa kali] diambil<br>
    d. Pengambilan karakter diambil hingga bertemu (tutup kurung)<br>

3. (?<=\(\p{N}+\)\s*)(.*)<br>
Digunakan untuk mengambil isi ayat<br>
Kondisi:<br>
    a. Text dimulai dengan (buka kurung)[nomor dapat diulang beberapa kali](tutup kurung) tanpa diambil (no ayat)<br>
    b. Setelah kondisi diatas semua karakter diambil<br>

In [16]:
for i in range(len(df_pasal)):
    no_ayat, isi_ayat = ambil_ayat(df_pasal["isi_pasal"][i])
    df_ayat = {"no_ayat": no_ayat, "isi_ayat": isi_ayat}
    display(pd.DataFrame(df_ayat))

,no_ayat,isi_ayat
0,,Dalam rangka mengembangkan upaya perlindungan ...


,no_ayat,isi_ayat
0,,Badan Perlindungan Konsumen Nasional berkedudu...


,no_ayat,isi_ayat
0,,Badan Perlindungan Konsumen Nasional mempunyai...


,no_ayat,isi_ayat
0,1,Untuk menjalankan fungsi sebgaimana dimaksud ...
1,2,Dalam melaksanakan tugas sebagaimana dimaksud...


In [17]:
no_ayat, isi_ayat = ambil_ayat(df_pasal["isi_pasal"][3])
df_ayat = {"no_ayat": no_ayat, "isi_ayat": isi_ayat}

In [18]:
df_ayat = pd.DataFrame(df_ayat)

# Penjelasan

In [19]:
penjelasan

'PRESIDEN\nREPUBLIK INDONESIA\nPENJELASAN\nATAS\nUNDANG-UNDANG REPUBLIK INDONESIA\nNOMOR 8 TAHUN 1999\nTENTANG\nPERLINDUNGAN KONSUMEN\nI. UMUM\nPembangunan dan perkembangan perekonomian umumnya dan khususnya di\nbidang perindustrian dan perdagangan nasional telah menghasilkan berbagai variasi\nbarang dan/atau jasa yang dapat dikonsumsi. Di samping itu, globalisasi dan\nperdagangan bebas yang didukung oleh kemajuan teknologi telekomunikasi dan\ninformatika telah memperluas ruang gerak arus transaksi barang dan/atau jasa\nmelintasi batas-batas wilayah suatu negara, sehingga barang dan/atau jasa yang\nditawarkan bervariasi baik produksi luar negeri maupun produksi dalam negeri.\nKondisi yang demikian pada satu pihak mempunyai manfaat bagi konsumen\nkarena kebutuhan konsumen akan barang dan/atau jasa yang diinginkan dapat\nterpenuhi serta semakin terbuka lebar kebebasan untuk memilih aneka jenis dan\nkualitas barang dan/atau jasa sesuai dengan keingingan dan kemampuan konsumen.\nDi sisi la

## Ambil Pasal

In [20]:
def ambil_penjelasan_pasal(text):
    pattern = "(?<=\\n|^|\s)(Pasal\s*\p{N}+.*?)(?=\\nPasal \p{N}|\sPasal \p{N}|$)"
    pasal = re.findall(pattern, text, re.DOTALL)
    if pasal:
        pattern_no = "(?<=Pasal)(?:\s*)(\p{N}+)"
        pattern_isi = "(?<=Pasal)(?:\s*\p{N}+\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) for i in pasal]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) for i in pasal]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

In [21]:
no_penjelasan_pasal, isi_penjelasan_pasal = ambil_penjelasan_pasal(penjelasan)
df_penjelasan_pasal = {'no_penjelasan_pasal': no_penjelasan_pasal, 'isi_penjelasan_pasal': isi_penjelasan_pasal}

In [22]:
df_penjelasan_pasal = pd.DataFrame(df_penjelasan_pasal)

## Ambil Ayat

In [23]:
df_penjelasan_pasal['isi_penjelasan_pasal'][61]

'Ayat (1)\nCukup jelas\nAyat (2)\nCukup jelas\nAyat (3)\nCukup jelas'

In [24]:
def ambil_penjelasan_ayat(text):
    pattern = "(?<=\\n|^|\s)(Ayat\s*\(\p{N}+\).*?)(?=\\nAyat\s*\(\p{N}+\)|\sAyat\s*\(\p{N}+\)|$)"
    ayat = re.findall(pattern, text, re.DOTALL)
    if ayat:
        pattern_no = "(?<=Ayat)(?:\s*\()(\p{N}+)"
        pattern_isi = "(?<=Ayat)(?:\s*\(\p{N}+\)\s*\\n)(.*)"
        no = [re.search(pattern_no, i, re.DOTALL).group(1) for i in ayat]
        isi = [re.search(pattern_isi, i, re.DOTALL).group(1) for i in ayat]
        return no, isi
    else:
        no = [""]
        isi = [text]
        return no, isi

In [25]:
for i in range(len(df_penjelasan_pasal)):
    no_penjelasan_ayat, isi_penjelasan_ayat = ambil_penjelasan_ayat(df_penjelasan_pasal['isi_penjelasan_pasal'][i])
    df_penjelasan_ayat = {'no_penjelasan_ayat': no_penjelasan_ayat, 'isi_penjelasan_ayat': isi_penjelasan_ayat}
    display(pd.DataFrame(df_penjelasan_ayat))

,no_penjelasan_ayat,isi_penjelasan_ayat
0,,Angka 1\nCukup jelas\nAngka 2\nDi dalam kepust...


,no_penjelasan_ayat,isi_penjelasan_ayat
0,,Perlindungan konsumen diselenggarakan sebagai ...


,no_penjelasan_ayat,isi_penjelasan_ayat
0,,Cukup jelas


,no_penjelasan_ayat,isi_penjelasan_ayat
0,,Huruf a\nCukup jelas \nHuruf b\nCukup jelas\nH...


,no_penjelasan_ayat,isi_penjelasan_ayat
0,,Cukup jelas


...

,no_penjelasan_ayat,isi_penjelasan_ayat
0,,Cukup jelas


,no_penjelasan_ayat,isi_penjelasan_ayat
0,1,Cukup jelas
1,2,Cukup jelas
2,3,Cukup jelas


,no_penjelasan_ayat,isi_penjelasan_ayat
0,,Cukup jelas


,no_penjelasan_ayat,isi_penjelasan_ayat
0,,Cukup jelas


,no_penjelasan_ayat,isi_penjelasan_ayat
0,,Cukup jelas\nTAMBAHAN LEMBARAN NEGARA REPUBLIK...


In [26]:
no_penjelasan_ayat, isi_penjelasan_ayat = ambil_penjelasan_ayat(df_penjelasan_pasal['isi_penjelasan_pasal'][61])
df_penjelasan_ayat = {'no_penjelasan_ayat': no_penjelasan_ayat, 'isi_penjelasan_ayat': isi_penjelasan_ayat}

In [27]:
df_penjelasan_ayat = pd.DataFrame(df_penjelasan_ayat)